In [27]:
#Packages being downloaded
import csv
import pandas as pd
import re
import xml.etree.ElementTree as ET

from tkinter import filedialog
from tkinter import *

In [26]:
#%%python
def getFile(sdtFile):
    root = Tk()
    root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("text files","*.csv"),("all files","*.*")))
    changePath(root.filename,sdtFile)
    root.destroy()
    return(root.filename)

In [25]:
def changePath(path,sdtFile):
    tree = ET.parse(sdtFile)
    root = tree.getroot()
    
    for elem in root.findall('.//Datasources/Datasource/Host'):
        elem.text=path
    
    for elem in root.findall('.//Datasources/Datasource/Name'):
        elem.text=path
        
    tree.write(sdtFile,encoding="utf-8", xml_declaration=True)

In [24]:
def readCSV(csvName):
    df = pd.read_csv(csvName)
    print("reading csv done")
    return df

In [23]:
def getCat(df,colorName):
    x=0
    for name in df.columns:
        if df.dtypes[x] == 'object':
             if len(df[name].astype('category').cat.categories.tolist()) >1 and len(df[name].astype('category').cat.categories.tolist()) <6 and colorName=="": 
                colorName=df.columns[x]
        x= x+1
    return colorName

In [22]:
def getCatCol(df,colorName):
    return df[colorName].astype('category').cat.categories.tolist()

In [21]:
def typeDefiner(df):
    colList = df.columns
    colList
    x = 0
    colNames = []
    for name in colList:
        if df.dtypes[x] == 'object':# or df.dtypes[x] == 'int64':
            pass
        else:
            colNames.append(colList[x])
                
        x = x + 1
    print("objects removed")
    print(colNames)
    return colNames

In [20]:
def toNumeric(sdtFile):

    import xml.etree.ElementTree as ET
    #CLEARS COLOR OUT NOW WE CAN INSERT ANYTHING
    file = ET.parse(sdtFile)
    root = file.getroot()
    for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
        elem.clear()

    for parent in root.findall('.//KeyValuePairs/..'):
        for element in parent.findall('KeyValuePairs'):
            parent.remove(element)
   #Try to write here
    for elem in root.findall('.//Color/RGB'):
        check_req_elems = elem.find('.//Function[@type="Text Field To Value"]')
        check_req_elems.type = "Linear Interpolation"
        check_req_elems.set("type","Linear Interpolation")
        attrib = {'type': 'BoundField'}
        minmax = ET.SubElement(check_req_elems, 'MinMax', attrib)
        minmax.text= "\n\t\t\t\t\t\t\t"
        min = ET.SubElement(minmax, 'Min')
        min.text = '0'
        min.tail="\n\t\t\t\t\t\t"
        minmax.tail="\n\t\t\t\t\t"
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)
    print("color fixed")

In [40]:
def XMLParser(colNames, sdtFile,colorName):
    print(colNames)

    tree = ET.parse(sdtFile)
    root = tree.getroot()
    count=0;
    print(len(colNames))
    for elem in root.iter('InputField'):
        print(count)
        #count+=1;
        #check_req_elem = elem.find('.//InputField')
        if elem.get('name')!="color":
            elem.set('field', colNames[count])
            count+=1
        else:
            elem.set('field', colorName)
        
    #tree.write('New DAL 900.sdt')
    tree.write('emptySDT.sdt')
    print("XML updated")

In [30]:
def runAll(sdtFile):
    colorCol= None
    colorName=""
    csv = getFile(sdtFile)
    df = readCSV(csv)
    colorName=getCat(df,colorName)
    print(colorName)
    colorCol=getCatCol(df,colorName)
    colNames = typeDefiner(df)
    colNames.append(colorName)
    XMLParser(colNames, sdtFile,colorName)
    setupColor(colorCol, sdtFile)
   # csv = getFile()
#readCSV(csv)

In [42]:
#%%python
runAll('emptySDT.sdt')

reading csv done
Gender
objects removed
['Height', 'Age', 'GPA']
['Height', 'Age', 'GPA', 'Gender']
4
0
1
2
3
XML updated
23,208,185
191,152,10
165,239,184
50,170,147
58,171,173


# Extra functions

In [4]:
def elevationFinder(df):
    if df.columns.contains("elevation") or df.columns.contains("Elevation") or df.columns.contains("height"):
        longitude = df["Longitude"]
    else:
        longitude = "none found"
    return longitude

In [3]:
def latFinder2(df):
    x = 0
    for columns in df.columns:
        searchObj = re.match( r'LAT', df.columns[x], re.I)
        if searchObj:
            #print("found it")
            lat = df.columns[x]
            return lat
        else:
            x = x + 1
            pass

In [2]:
def longFinder2(df):
    x = 0
    for columns in df.columns:
        searchObj = re.match( r'LONG', df.columns[x], re.I)
        if searchObj:
            #print("found it")
            long = df.columns[x]
            return long
        else:
            x = x + 1
            pass

In [38]:
def setupColor(colorCol,sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    x=0
    if(colorCol==None):#and there are other numeric options
        toNumeric(sdtFile)
    else:
        for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
            for element in elem.findall('KeyValuePair/Key'):
                element.text=colorCol[x]
                x=x+1

            if(len(colorCol)>2):
                while x<len(colorCol):
                    for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]'):
                        pairs = elem.find('.//KeyValuePairs')
                #fixes spacing so new pair will be at right column/indent area
                        for child in pairs.findall('.//KeyValuePair'):
                            child.tail="\n\t\t\t\t\t\t\t"
                        keyVal =ET.SubElement(pairs, 'KeyValuePair')
                        keyVal.text="\n\t\t\t\t\t\t\t\t"
                        keyVal.tail="\n\t\t\t\t\t\t"
                        keyValP= ET.SubElement(keyVal, 'Key')
                        keyValP.text=colorCol[x]
                        keyValP.tail="\n\t\t\t\t\t\t\t\t"
                        key = ET.SubElement(keyVal, 'Value')
                    
                        key.tail="\n\t\t\t\t\t\t\t"
                        x=x+1
    
    x=0
    import random

    count=0
    colorList=[] 
    while count<len(colorCol):
        randomCol(colorList)
        count=count+1


    for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
            for element in elem.findall('KeyValuePair/Value'):
                element.text=colorList[x]
                print(colorList[x])
                x=x+1
                
                
    file.write('emptySDT1.sdt',encoding="utf-8", xml_declaration=True)


In [14]:
def randomCol(colorList):
    r = random.random()*255
    r=int(r)
    g = random.random()*255
    g=int(g)
    b = random.random()*255
    b=int(b)
    color=str(r)+","+str(g)+","+str(b)
    colorList.append(color)

In [15]:
def resetToDefault(sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    
    

['12,176,68', '157,173,26', '42,161,173', '205,30,198', '23,132,184', '122,172,228', '180,36,218', '60,238,113', '41,202,157', '217,66,9', '158,207,62', '218,174,156', '58,13,177', '2,27,95', '160,24,177', '65,76,147', '90,188,84', '25,172,47', '88,168,71', '49,49,10', '248,83,110', '252,38,143', '12,177,190', '129,177,78', '245,40,166', '232,152,26', '36,101,4', '23,66,63', '220,199,15', '148,61,113', '228,32,203', '144,152,87', '105,54,20', '211,170,58', '27,220,102', '214,23,154', '151,74,44', '221,154,110', '60,184,1', '198,154,86', '56,11,83', '125,4,86', '238,23,193', '220,134,22', '84,46,171', '186,49,175', '65,100,85', '120,19,254', '164,154,65', '136,130,244', '177,164,181', '230,245,188', '163,143,46', '27,95,41', '229,205,205', '158,34,88', '105,146,189', '41,35,69', '151,1,106', '132,215,160', '53,80,55', '173,225,174', '137,161,231', '125,246,123', '99,20,196', '57,190,90', '0,71,167', '211,122,242', '214,53,107', '11,120,135', '6,132,227', '212,56,24', '135,126,235', '96,